# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part1_file = "weather_df.csv"

types_df = pd.read_csv(part1_file)

vacation_data_df = types_df.drop(columns=["Unnamed: 0"])
vacation_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,bluff,100,NZ,1618266886,95,-46.6000,168.3333,54.00,1.99
1,hobart,75,AU,1618266982,87,-42.8794,147.3294,46.99,6.91
2,manbij,0,SY,1618267153,87,36.5281,37.9549,37.40,5.75
3,hukuntsi,0,BW,1618267153,36,-24.0000,21.7500,69.13,8.97
4,tondano,20,ID,1618267154,94,1.3038,124.9112,75.20,1.97
...,...,...,...,...,...,...,...,...,...
531,vitre,0,FR,1618267064,70,48.1178,-1.2058,41.00,3.44
532,petropavlovka,1,RU,1618267339,65,50.6114,105.3196,28.62,2.48
533,uyuni,31,BO,1618267339,44,-20.4597,-66.8250,56.68,4.27
534,gazojak,71,TM,1618267339,23,41.1875,61.4036,58.98,8.79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = vacation_data_df[["Lat", "Lng"]]

humidity = vacation_data_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_vacation_df = pd.DataFrame(vacation_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_vacation_df["Max Temp"] <= 80) & (new_vacation_df["Max Temp"] >= 70)
wind_speed = new_vacation_df["Wind Speed"] < 10
cloudiness = new_vacation_df["Cloudiness"] == 0

new_vacation_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
94,saint-philippe,78.80,6.91,0
135,saint-joseph,78.80,6.91,0
150,itacare,75.20,3.44,0
369,jiwani,78.48,3.33,0
391,saint-pierre,78.80,6.91,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(vacation_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,bluff,NZ,-46.6000,168.3333,
1,hobart,AU,-42.8794,147.3294,
2,manbij,SY,36.5281,37.9549,
3,hukuntsi,BW,-24.0000,21.7500,
4,tondano,ID,1.3038,124.9112,
...,...,...,...,...,...
531,vitre,FR,48.1178,-1.2058,
532,petropavlovka,RU,50.6114,105.3196,
533,uyuni,BO,-20.4597,-66.8250,
534,gazojak,TM,41.1875,61.4036,


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        
        print("MISSING RESULT. SKIPPING...")
        
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -46.6129616,
                    "lng": 168.3576089
                },
                "viewport": {
                    "northeast": {
                        "lat": -46.61168132010727,
                        "lng": 168.3590292798927
                    },
                    "southwest": {
                        "lat": -46.61438097989272,
                        "lng": 168.3563296201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Land's End Boutique Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2160,
                    "html_attributions": [
                 

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLSpUDbb78UT8SbWjhOj720mPme7vgpBE762z2yjCM7S0XAj6rRtTAGnxd-_t0WHKEDylCTOJTKgaThSgfg47FjkkB9OaPBG8frUNREjBicoJaS1UGkXFcDg1T7UBLu983ALmfoe8ezD-C_GBPeUPk09zntqhC8DqXHtcrDldy8U5ZtzdwKOZbAz5zU4VklydbkxVAW_TeZzNWc9IXZReou87Jcm_AyJ33X46UiMjaMyaLcI3qRUOqUckique-dwoy5z1dSMeXRYFJrJcGzzOpNAldNt8NPLl_T9SwNhJg31E5Ec9bh6JCSMzP4NSB4dAMsX_M7woatBB2s0thpr0p84zgYBOe3d-nK-HsJC7Ft8Ym5kTuUAE9Kq2YAGTeJytRT7V7vpGc9_Cs4jGR_YHH_cDmEvEL_dbU8aPHOWeSWwTW1LDyuAL6q1aGmrB4",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -42.8847481,
                    "lng": 147.3300213
                },
                "viewport": {
                    "northeast": {
                        "lat": -42.88334767010728,
                        "lng": 147.3314094798928
                    },
                    "southwest": {
                        "lat": -42.886047

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 36.5271265,
                    "lng": 37.9511156
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.52842107989272,
                        "lng": 37.95249667989272
                    },
                    "southwest": {
                        "lat": 36.52572142010728,
                        "lng": 37.94979702010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Manbej City Hotel",
            "photos": [
                {
                    "height": 540,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/103107425366789321503\">A Google User

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
MISSING RESULT. SKIPPING...
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 69.7041553,
                    "lng": 170.282609
                },
                "viewport": {
                    "northeast": {
                        "lat": 69.70572057989271,
                        "lng": 170.2838556798928
                    },
                    "southwest": {
                        "lat": 69.70302092010726,
                        "lng": 170.2811560201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Munitsipal'naya Gostinitsa",
            "photos": [
                {
                    "height": 1536,
                    "html_attribu

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
MISSING RESULT. SKIPPING...
{
    "html_attributions": [],
    "next_page_token": "ATtYBwJO8IVcL4I0WHKaeJe1ykQDirmpUAAZcFGXpjmF3MNTFfnsMeLIpq6lPtD_jao-RX9F79unBh02J7WLgVHFOKLxo4Qoag4n-3s3Z0T1SIbLa8R9ke5avd4uPGTgxL95NIc0YlmRnTU_LwNySJlO7_054DS5ZrufDla65sRDHok5y6VVnf15XnZ_fevXIZzu6y1x61P9I64kQj9x1R2gOuiaunlJkof3WW90fYySMly1Ch5UpNo1wm5CTHpf_3G7i_zPAXeXeJExQ9CGQBcoUICeFopzUouFNKoPty4q4X-qQtsMf_QpHk-ti9NE-ya8EaPWn08TC6VeUCcWUQYnOXgFReisb6aUZTxGnncSpZzJhgvGogq3SKGIIzPHs1EEgYvWfQfJ65e6BBx6pX_T6SxYzNowhCK8r79C2PaTwe-Zfg2PKx_3PHfpvlxlEN4",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 38.6998556,
                    "lng": -9.417411099999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.70122162989272,
                        "lng": -9.

...

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
MISSING RESULT. SKIPPING...
{
    "html_attributions": [],
    "next_page_token": "ATtYBwKoRbk7U4UzwGnmWdX7bFTLqunA0MamLX_RVUmN1kIHEyLFCCULHUK-qNnUA7zOw9S0ONL46NkuTAuvoKrPnYJmuJRZhiHLkzhqyc98Q50H5wORuKHYhMYYYHUuau6b1HRdsIIDhtIDZzt2GgQYAPdvhxiuWzhrqpXT9xkY6YH2L7MS_L6drZT_hREsMGbu1U3Q_1gnRL8CHQJPxiiKkQ50jKzOcG__AvJxLENAhk_sr0yM6VoQU5WiYgkTnQyhF0AfCC-lyhy4fvEgbzYAd4bHAdHEGyUtVTHTmqTFoKya5lbg3LxjIm-HFhRW7DTNJVqIjclp1dNhhQjSy0YZvZZV-bh95bwSz7OY7zWxwrIlkvmFlc21mtcUMZmdTiCAXvosCx6Ncehp9_FTJSWB5FaGbyPBaqp0g8YeSpW6BWoaF19qY0FlV1guiTemfzo",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -45.874947,
                    "lng": 170.50612
                },
                "viewport": {
                    "northeast": {
                        "lat": -45.87354647010729,
                        "lng": 170.5075311

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 71.6922089,
                    "lng": 128.871611
                },
                "viewport": {
                    "northeast": {
                        "lat": 71.69353682989272,
                        "lng": 128.87235585
                    },
                    "southwest": {
                        "lat": 71.69083717010729,
                        "lng": 128.86937645
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Arktika",
            "photos": [
                {
                    "height": 3000,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/103359888378006991889\">\u0420\u043e\u043c\u0430 \u041a\

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLc-DMt3uVtMXaF8w2Xjx7aEA6CZuXEDqRXqqjnBWUzHRNNnhWMNh9mYCvyIfee7_5S-FRm0Be859DLKPcJq570XajNZBdOYoRsSENr43D3Gf2hFYEYvg9mTkzvGg4pZNzoCAqRlZNrgqzQ0GuY308Cj8gWaWC_bGltMraQYoZIiFmM5W5iQW4NAQbPkzuJPbxFlg4gQ2KqNiQOOrJJ6mLMaX11igSjNde3TYH4HZizoQKcyNh2IV40qmLfjMOsoSe2tNa-ennyW40lxO4QhTuDR1UMt8H8lZjhs2vHeNdA674GgInbfY8BKCbbqT1LRkqGtiNsmMZ0WCtppxBcDzJxy1QU5Cr9HL3Z_OU57EVVYIfwtZbR6W82p0fwHN3opw8hT_TKK1Sf-3NWOFagAMYLRaWgSbPqetnarkW8qo69BewLcyXEqTHEX4Ww_Q4",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -45.5725389,
                    "lng": -72.07082609999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -45.57117192010728,
                        "lng": -72.06953612010727
                    },
                    "southwest": {
                        "lat": -4

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 65.80129769999999,
                    "lng": 87.9712102
                },
                "viewport": {
                    "northeast": {
                        "lat": 65.80269492989271,
                        "lng": 87.97341985
                    },
                    "southwest": {
                        "lat": 65.79999527010726,
                        "lng": 87.97047365000002
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png",
            "name": "\u041e\u041e\u041e \"\u0422\u0443\u0440\u0443\u0445\u0430\u043d\u0441\u043a\u0430\u044f \u043f\u0443\u0448\u043d\u0438\u043d\u0430\"",
            "opening_hours": {
                "open_now": false
            },
            "photos": [
        

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLbYkYO-hJm3zlANbtJEmVWcbgJXI34YT_YwMEAdRPY7M7ojXq9hZmyW0_XRjYau69EnI7WB5dbuOp7GWHSIhABPSuSWHIXNJVz8B0bUb5Nv-5oAQ5r6P03Ew7nAfcyryWsBb7rLQjO68YG-P888mrM2kLaFkgEH9VoFrTfGo7wnI2tZGHPCY0eRScT9lrnBpbeTc63g0aDaEBTe_CUG7wWV8_E5WBHUF6sLcJSlFyFnsbS8GvpBLRzJmz2msAh3n3ibwN4D19LWwJ-w6eZ27FTs9c45dHZB_6sion5XCy-2UsA3i3gg_vIrkmevoutVXWK6LV0h83_5rdoRPyQYypW2xo7BhjDcqo3xV6fTtEDN0-0MDejDn0PScyfyk11vBQzavVX9hY86qa8vK6Ho4COL6S4L5TGTmqwTc--F-xkmN2qJR86eQyPZt_ZZg0",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -13.7977882,
                    "lng": -47.4523442
                },
                "viewport": {
                    "northeast": {
                        "lat": -13.79642912010728,
                        "lng": -47.45092047010728
                    },
                    "southwest": {
                        "lat": -13.79912

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig